In [115]:
from pydsstools.heclib.dss import HecDss
from pydsstools.core import TimeSeriesContainer
import pandas as pd
import numpy as np
# import plotly
from plotly.graph_objs import Scatter, Layout
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio


In [48]:
# eventList = ['TS_Bill', 'Hurricane_Katrina', 'Hurricane_Isaac']

# event = eventList[0]

reservoir_params = ['FLOW', 'FLOW-COMBINE', 'ELEVATION']
gage_params = ['FLOW']
gage_element = 'Oberlin'
reservoir_element = 'Mill Creek Reservoir'

In [49]:
# Open the DSS file and get the events from the f-part of the paths.
dss_file = HecDss.Open(r"Z:\MillCreek\MillCreek_HMS\MillCreek_HMS.dss")
pathname_pattern = "//*/ELEVATION/*/1hour/*/"
path_list = dss_file.getPathnameList(pathname_pattern, sort=1)
# path_list

In [50]:
# Get f part of the pathnames
f_part = [path.split('/')[-2] for path in path_list]
f_part = list(set(f_part))
# drop any string element containing 'Calibration' or 'Validation'.
f_part = [f for f in f_part if 'Calibration' not in f and 'Validation' not in f]
f_part

['RUN:Storm10local_Res_Qlow_EL77',
 'RUN:Storm100local_Res_Qlow_EL77',
 'RUN:Storm100local_Res_Qlow_EL75',
 'RUN:Storm10all_noRes_Qhigh',
 'RUN:Storm100local_noRes_Qlow',
 'RUN:Storm10all_Res_Qhigh_EL77',
 'RUN:Storm100all_noRes_Qhigh',
 'RUN:Storm10local_Res_Qlow_EL75',
 'RUN:Storm10all_Res_Qhigh_EL75',
 'RUN:Storm100all_Res_Qhigh_EL77',
 'RUN:Storm100all_Res_Qhigh_EL75',
 'RUN:Storm10local_noRes_Qlow']

In [78]:
# Pair up the f_parts that we want plotted together.
reservoir_pairings = {
    "Mill Creek Reservoir 10Year Rainfall": [
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm10local_noRes_Qlow', 
            'name': 'No Reservoir'
        }
    ],
    "Mill Creek Reservoir 100Year Rainfall": [
        {
            'fpart': 'RUN:Storm100local_Res_Qlow_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm100local_Res_Qlow_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm100local_noRes_Qlow', 
            'name': 'No Reservoir'
        }
    ]
}
gage_pairings = {
    "Oberlin 10yr (Mill Creek Rainfall Only)": [
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm10local_noRes_Qlow', 
            'name': 'No Reservoir'
        }
    ],
    "Oberlin 10yr (Basin Wide Rainfall)": [
        {
            'fpart': 'RUN:Storm10all_Res_Qhigh_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm10all_Res_Qhigh_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm10all_noRes_Qhigh', 
            'name': 'No Reservoir'
        }
    ],
    "Oberlin 100yr (Mill Creek Rainfall Only)": [
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm10local_Res_Qlow_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm100local_noRes_Qlow', 
            'name': 'No Reservoir'
        }
    ],
    "Oberlin 100yr (Basin Wide Rainfall)": [
        {
            'fpart': 'RUN:Storm100all_Res_Qhigh_EL75', 
            'name': '75'
        },
        {
            'fpart': 'RUN:Storm100all_Res_Qhigh_EL77', 
            'name': '77'
        },
        {
            'fpart': 'RUN:Storm100all_noRes_Qhigh', 
            'name': 'No Reservoir'
        }
    ]
}

In [126]:
# Get pathnames for each pairing
for i, plot_title in enumerate(gage_pairings):

    # Setup plotly subplot figure
    fig = make_subplots(  
        rows=1, 
        cols=1,
        # vertical_spacing=3,
        # padding = 
        # vertical_spacing=(1 / (n - 1)),
        #subplot titles are each pairing key.
        # subplot_titles=['flowation', 'InFlow and OutFlow'],
        
    )
    
    for curve in gage_pairings[plot_title]:
        # print(plot_title, curve['name'], curve['fpart'])
        # get flowation pathnames
        flow_pattern = f"//{gage_element}/FLOW/*/1hour/{curve['fpart']}/"
        flow_paths = dss_file.getPathnameList(flow_pattern, sort=1)
        # remove d-part from pathnames and remove duplicates
        flow_paths = [path.replace(path.split('/')[4],"") for path in flow_paths]
        if len(flow_paths) > 0:
            flow_path = flow_paths[0]
        else:
            flow_path = None
        # print('flowation: ', flow_path)
        curve['flow_path'] = flow_path

        # get path data
        if flow_path:
            ts = dss_file.read_ts(flow_path)
            df = pd.DataFrame()
            df['Times'] = ts.pytimes
            df['Values'] = ts.values
            df['Missing'] = ts.nodata
            df.Values = np.where(df.Missing == True, np.NaN, df.Values)
            
            # add trace to subplot

            fig.add_trace(
                go.Scatter(
                    x=df['Times'], 
                    y=df['Values'], 
                    mode='lines',
                    name=f"{curve['name']}"
                ),
                row=1, 
                col=1
            )
        
        

    # reservoir_pairings
    fig.update_layout(
        title=plot_title,
        yaxis_title="Flow (cfs)",
        # showlegend=False,
        font=dict(
            family="Arial",
            size=18,
            color="RebeccaPurple"
        )
    )
    pio.write_image(fig, file=f"./results/millCreek/{plot_title}.png", width=1600, height=1000)
    fig.show()

In [128]:
# Get pathnames for each pairing
for i, plot_title in enumerate(reservoir_pairings):

    # Setup plotly subplot figure
    fig = make_subplots(  
        rows=2, 
        cols=1,
        # vertical_spacing=3,
        # padding = 
        # vertical_spacing=(1 / (n - 1)),
        #subplot titles are each pairing key.
        subplot_titles=['Elevation', 'Inflow and OutFlow'],
        
    )
    
    for curve in reservoir_pairings[plot_title]:
        # print(plot_title, curve['name'], curve['fpart'])
        # get elevation pathnames
        elev_pattern = f"//{reservoir_element}/ELEVATION/*/1hour/{curve['fpart']}/"
        elev_paths = dss_file.getPathnameList(elev_pattern, sort=1)
        # remove d-part from pathnames and remove duplicates
        elev_paths = [path.replace(path.split('/')[4],"") for path in elev_paths]
        if len(elev_paths) > 0:
            elev_path = elev_paths[0]
        else:
            elev_path = None
        # print('elevation: ', elev_path)
        curve['elev_path'] = elev_path

        # get path data
        if elev_path:
            ts = dss_file.read_ts(elev_path)
            df = pd.DataFrame()
            df['Times'] = ts.pytimes
            df['Values'] = ts.values
            df['Missing'] = ts.nodata
            df.Values = np.where(df.Missing == True, np.NaN, df.Values)
            
            # add trace to subplot

            fig.add_trace(
                go.Scatter(
                    x=df['Times'], 
                    y=df['Values'], 
                    mode='lines',
                    name=f"Elevation {curve['name']}"
                ),
                row=1, 
                col=1
            )
        
        # get inflow pathnames
        inflow_pattern = f"//{reservoir_element}/FLOW-COMBINE/*/1hour/{curve['fpart']}/"
        inflow_paths = dss_file.getPathnameList(inflow_pattern, sort=1)
        # remove d-part from pathnames and remove duplicates
        inflow_paths = [path.replace(path.split('/')[4],"") for path in inflow_paths]
        if len(inflow_paths) > 0:
            inflow_path = inflow_paths[0]
        else:
            inflow_path = None
        # print('inflow: ', inflow_path)
        curve['inflow_path'] = inflow_path

        # get path data
        if inflow_path:
            ts = dss_file.read_ts(inflow_path)
            df = pd.DataFrame()
            df['Times'] = ts.pytimes
            df['Values'] = ts.values
            df['Missing'] = ts.nodata
            df.Values = np.where(df.Missing == True, np.NaN, df.Values)
            
            # add trace to subplot

            fig.add_trace(
                go.Scatter(
                    x=df['Times'], 
                    y=df['Values'], 
                    mode='lines',
                    name=f"Inflow {curve['name']}"
                ),
                row=2, 
                col=1
            )

        # get outflow pathnames
        outflow_pattern = f"//{reservoir_element}/FLOW/*/1hour/{curve['fpart']}/"
        outflow_paths = dss_file.getPathnameList(outflow_pattern, sort=1)
        # remove d-part from pathnames and remove duplicates
        outflow_paths = [path.replace(path.split('/')[4],"") for path in outflow_paths]
        if len(outflow_paths) > 0:
            outflow_path = outflow_paths[0]
        else:
            outflow_path = None
        # print('outflow: ', outflow_path, '\n\n')
        curve['outflow_path'] = outflow_path

        # get path data
        if outflow_path:
            ts = dss_file.read_ts(outflow_path)
            df = pd.DataFrame()
            df['Times'] = ts.pytimes
            df['Values'] = ts.values
            df['Missing'] = ts.nodata
            df.Values = np.where(df.Missing == True, np.NaN, df.Values)
            
            # add trace to subplot

            fig.add_trace(
                go.Scatter(
                    x=df['Times'], 
                    y=df['Values'], 
                    mode='lines',
                    name=f"Outflow {curve['name']}"
                ),
                row=2, 
                col=1
            )

    # reservoir_pairings
    fig.update_layout(
        title=plot_title,
        # autosize=False,
        height=1000, 
        width=1800,
        # template="plotly_dark",
        # xaxis_title="Time",
        yaxis_title="Elevation (ft)",
        # xaxis2_title="Time",
        yaxis2_title="Flow (cfs)",
        # showlegend=False,
        font=dict(
            family="Arial",
            size=18,
            color="RebeccaPurple"
        )
    ).write_image(f"./results/millCreek/{plot_title}.png")
    fig.show()

In [ ]:
gageList = []
DSSpaths_List = []
DSSpaths_List_sim = []

for path in path_list:
    
    pathSplit = path.split("/")
    gage = pathSplit[2]
    cPart = pathSplit[3]
    ePart = pathSplit[5]
    fPart = pathSplit[6]
    gageList.append(gage)
    
    # Remove d-part
    noDpartPath = f"//{gage}/{cPart}//{ePart}/{fPart}/"
    noDpartPath_sim = f"//{gage}/FLOW//{ePart}/{fPart}/"
    DSSpaths_List.append(noDpartPath)
    DSSpaths_List_sim.append(noDpartPath_sim)

# sort the lists alphabetically, and remove dups
gageList = sorted(list(set(gageList)))
DSSpaths_List = sorted(list(set(DSSpaths_List)))
DSSpaths_List_sim = sorted(list(set(DSSpaths_List_sim)))

In [ ]:
# Setup plotly subplot figure
fig = make_subplots(  
    rows=len(gageList), 
    cols=1, 
    # vertical_spacing=3,
    # padding = 
    # vertical_spacing=(1 / (n - 1)),
    subplot_titles=gageList,
    
)

In [ ]:
for i, path_obs in enumerate(DSSpaths_List):
    
    # Create sim path based on FLOW-OBSERVED path.
    path_sim = path_obs.replace("FLOW-OBSERVED", "FLOW")

    # Make plot name = bPart
    pathSplit = path_obs.split("/")
    gage = pathSplit[2]

    ts_obs = fid.read_ts(path_obs)
    ts_sim = fid.read_ts(path_sim)

    # setup dataframe of traces
    df_obs = pd.DataFrame()
    df_sim = pd.DataFrame()
    df_obs['Times_obs'] = ts_obs.pytimes
    df_obs['Values_obs'] = ts_obs.values
    df_obs['Missing_obs'] = ts_obs.nodata
    df_sim['Times_sim'] = ts_sim.pytimes
    df_sim['Values_sim'] = ts_sim.values
    df_sim['Missing_sim'] = ts_sim.nodata

    df_obs.Values_obs = np.where(df_obs.Missing_obs == True, np.NaN, df_obs.Values_obs)
    df_sim.Values_sim = np.where(df_sim.Missing_sim == True, np.NaN, df_sim.Values_sim)
    # df['sim'] = sim
    
    # concatenate dataframes for each sim to a single dataframe that will be output to a csv file
    # df_gage = pd.concat([df_gage,df])


    fig.append_trace(
        go.Scatter(
            x=df_obs.Times_obs, 
            y=df_obs.Values_obs,
            # name = sim,
            name = f'{gage} Flow Observed',
            legendgroup=f'{i}',
            # showlegend = False,
        ), 
        row=i+1,
        # row = 1, 
        col=1
    )
    fig.append_trace(
        go.Scatter(
            x=df_sim.Times_sim, 
            y=df_sim.Values_sim,
            # name = sim,
            name = f'{gage} Flow Simulated',
            legendgroup=f'{i}',
            # showlegend = False,
        ), 
        row=i+1,
        # row = 1, 
        col=1
    )

    fig_iso = make_subplots(  
        rows=1, 
        cols=1, 
        # vertical_spacing=3,
        # padding = 
        # vertical_spacing=(1 / (n - 1)),
        # subplot_titles=gage
    )

    fig_iso.append_trace(
        go.Scatter(
            x=df_obs.Times_obs, 
            y=df_obs.Values_obs,
            name = f'{gage} Flow Observed',
        ), 
        row=1,
        col=1
    )
    fig_iso.append_trace(
        go.Scatter(
            x=df_sim.Times_sim, 
            y=df_sim.Values_sim,
            name = f'{gage} Flow Simulated',
        ), 
        row=1,
        col=1
    )

    fig_iso.update_layout(
        height=1000, 
        width=2300,
        showlegend=True, 
        title_text=f"Amite HMS: {event} - {gage} {parameter} ({units})",
        font=dict(
                family='sans-serif', size=30
        ),
        template= "seaborn",
        hoverlabel=dict(
            font=dict(
                family='sans-serif', size=22
            ),
            namelength= -1
        )
    )

    fig_iso.update_yaxes(automargin=True)
    fig_iso.write_html(f"results/html/{gage} {event} Results.html")
    fig_iso.write_image(f"results/png/{gage} {event} Results.png")

fid.close()

    

In [ ]:
fig.update_layout(
    height=len(gageList)*1000, 
    width=2300,
    showlegend=True,
    legend_tracegroupgap = 955,
    font=dict(
            family='sans-serif', size=30
    ),
    title_text=f"Amite HMS Calibration Points {event} - {parameter} ({units})",
    template= "seaborn",
    hoverlabel=dict(
        font=dict(
            family='sans-serif', size=22
        ),
        namelength= -1
    )
)

fig.update_yaxes(automargin=True)
fig.update_annotations(font_size=30)
fig.write_html(f"results/html/{event} Results.html")
fig.write_image(f"results/png/{event} Results.png")

In [ ]:
# Writing out Figure Captions for LWI Report
i = 1
eventList.reverse()
with open('output.txt', 'a') as f:
    for gage in gageList:
        for event in eventList:
            f.write(f'\nFigure C.{i}: {gage} {event} Calibration ')
            i+=1